In [15]:
import numpy as np
import pandas as pd
from datetime import datetime as dt

# data extraction layer
yanki_df = pd.read_csv('./raw data/yanki_ecommerce.csv')
# yanki_df.head()

In [158]:
## DATA CLEANING AND TRANSFORMATION
#yanki_df.info()

# drop missing values
yanki_df = yanki_df.dropna(subset=['Order_ID', 'Customer_ID', 'State']);
#yanki_df.info()


In [44]:
 # convert order date from string to datetime

yanki_df['Order_Date'] = pd.to_datetime(yanki_df['Order_Date'], format="%d/%m/%Y %H:%M")
#yanki_df.info() 

In [45]:
## NORMALISING THE DATA - As done in the ERD 
customer_df = yanki_df[['Customer_ID', 'Customer_Name', 'Email', 'Phone_Number']].copy().drop_duplicates().reset_index(drop=True)
#customer_df.head()

In [46]:
## PRODUCTS TABLE
products_df = yanki_df[['Product_ID', 'Product_Name', 'Brand', 'Category', 'Price']].copy().drop_duplicates().reset_index(drop=True)
#products_df.head()

In [47]:
## SHIPPING ADDRESS TABLE
shipping_df = yanki_df[['Customer_ID', 'Shipping_Address', 'City', 'State', 'Country', 'Postal_Code']].copy().drop_duplicates().reset_index(drop=True)

# Add shipping_ID column, use the index column
shipping_df.index.name = 'Shipping_ID'
shipping_df = shipping_df.reset_index()
#shipping_df.head()

In [48]:
## ORDER TABLE
orders_df = yanki_df[['Order_ID', 'Customer_ID', 'Product_ID', 'Quantity', 'Total_Price', 'Order_Date']].copy().drop_duplicates().reset_index(drop=True)
#orders_df

In [49]:
## PAYMENT METHODS TABLE
payments_df = yanki_df[['Order_ID', 'Payment_Method', 'Transaction_Status']].copy().drop_duplicates().reset_index(drop=True)
payments_df.index.name = 'Payment_ID'
payments_df = payments_df.reset_index()

#payments_df.head()

In [90]:
## SAVING FILES
customer_df.to_csv('./cleaned data/customers.csv', index = False);
products_df.to_csv('./cleaned data/products.csv', index = False);
shipping_df.to_csv('./cleaned data/shipping_address.csv', index = False);
orders_df.to_csv('./cleaned data/orders.csv', index = False);
payments_df.to_csv('./cleaned data/payments.csv', index = False);

In [37]:
# LOADING THE DATA INTO POSTGRESL
!pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------------------------- ------------ 0.8/1.2 MB 4.2 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 4.1 MB/s eta 0:00:00


In [38]:
import psycopg2

In [ ]:
# CREATE DATABASE CONNECTION
def connect_to_db():
    connection = psycopg2.connect(
        host = 'localhost',
        database = 'Yanki_Ecommerce_DB',
        user = 'postgres',
        password = 'sa'
    )
    return connection

con = connect_to_db()

<connection object at 0x0000026968D87010; dsn: 'user=postgres password=xxx dbname=Yanki_Ecommerce_DB host=localhost', closed: 0>

In [ ]:
# CREATE SQL TABLE

def create_tables():
    con = connect_to_db()
    cursor = con.cursor()
    queries = '''
                CREATE SCHEMA IF NOT EXISTS yanki;

                DROP TABLE IF EXISTS yanki.shipping_address;
                DROP TABLE IF EXISTS yanki.payments;
                DROP TABLE IF EXISTS yanki.orders;
                DROP TABLE IF EXISTS yanki.customers;
                DROP TABLE IF EXISTS yanki.products;
                


                CREATE TABLE yanki.customers(
                    Customer_ID UUID primary key,
                    Customer_Name varchar(255),
                    Email varchar(50),
                    Phone_Number TEXT
                );

                CREATE TABLE yanki.products(
                Product_ID UUID primary key,
                    Product_Name varchar(255),
                    Brand TEXT,
                    Category TEXT,
                    Price FLOAT
                );

                CREATE TABLE yanki.shipping_address(
                    Shipping_ID SERIAL primary key,
                    Customer_ID UUID,
                    Shipping_Address TEXT,
                    City TEXT,
                    State TEXT,
                    Country TEXT,
                    Postal_Code TEXT,
                    FOREIGN KEY (Customer_ID) REFERENCES yanki.customers(Customer_ID)
                );
    

                CREATE TABLE yanki.orders(
                    Order_ID UUID primary key,
                    Customer_ID UUID,
                    Product_ID UUID,
                    Quantity INT,
                    Total_Price FLOAT,
                    Order_Date DATE,
                    FOREIGN KEY (Customer_ID) REFERENCES yanki.customers(Customer_ID),
                    FOREIGN KEY (Product_ID) REFERENCES yanki.products(Product_ID)
                );


                CREATE TABLE yanki.payments(
                    Payment_ID SERIAL primary key,
                    Order_ID UUID,
                    Payment_Method TEXT,
                    Transaction_Status TEXT,
                    FOREIGN KEY (Order_ID) REFERENCES yanki.orders(Order_ID)
                );

            '''
    cursor.execute(queries);
    con.commit();
    cursor.close();
    con.close();

In [ ]:
create_tables();

In [ ]:
## LOAD DATA TO DATABASE
import csv

def clear_db_data():
    con = connect_to_db()
    cursor = con.cursor()
    cursor.execute('TRUNCATE yanki.products, yanki.shipping_address, yanki.payments, yanki.orders, yanki.customers;')
    con.commit()
    cursor.close()
    con.close()
    print('db clearing successful')

def load_data_from_csv(file_path):
    con = connect_to_db()
    cursor = con.cursor()

    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        
        # DYNAMIC INSERT
        table_name = file.name.split('/')[-1].split('.')[0]
        dt_columns = pd.read_csv(file_path).columns.tolist()
        placeholders = ', '.join(['%s'] * len(dt_columns))
        column_names = ', '.join(dt_columns)

        next(reader)
        for row in reader:
            cursor.execute(f'''
                INSERT INTO yanki.{table_name} ({column_names}) VALUES ({placeholders});
                ''', row
            )

        # MANUAL INSERT i.e one after the other
        # next(reader)
        # for row in reader:
        #     cursor.execute('''
        #         INSERT INTO yanki.payments (Payment_ID, Order_ID, Payment_Method, Transaction_Status) VALUES (%s, %s, %s, %s);
        #         ''', row
        #     )

    con.commit()
    cursor.close()
    con.close()



In [159]:
clear_db_data();

data_files = ['customers.csv', 'shipping_address.csv', 'products.csv', 'orders.csv', 'payments.csv']

for d in data_files:
    file_path = './cleaned data/'+d
    load_data_from_csv(file_path)
    print(f'{d} successfully loaded')

db clearing successful
customers.csv successfully loaded
shipping_address.csv successfully loaded
products.csv successfully loaded
orders.csv successfully loaded
payments.csv successfully loaded
